#### **Ingesta de la carpeta "production_country"**

In [0]:
dbutils.widgets.text("environment","")
var_environment = dbutils.widgets.get("environment")

In [0]:
dbutils.widgets.text("file_date","2024-12-30")
var_file_date = dbutils.widgets.get("file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Librerías

In [0]:
from pyspark.sql.functions import lit,current_timestamp
from pyspark.sql.types import StructType,StructField,IntegerType,StringType

##### Esquema

In [0]:
production_country_schema = StructType([
    StructField("movieId",IntegerType(),False),
    StructField("countryId",IntegerType(),False)
])

##### Leer ficheros JSON

In [0]:
df_production_country = spark.read.option("multiline", True).schema(production_country_schema).json(f"{bronze_folder_path}/{var_file_date}/production_country")

##### Renombrar y añadir columnas

In [0]:
df_production_country_final = add_ingestion_date(df_production_country).withColumnsRenamed({"movieId":"movie_id","countryId":"country_id"}) \
                                                   .withColumn("environment",lit(var_environment)).withColumn("file_date",lit(var_file_date))

##### Escribir en Silver

In [0]:
merge_condition = "target.movie_id = source.movie_id AND target.country_id = source.country_id AND target.file_date = source.file_date"
merge_delta_lake(df_production_country_final,"movie_silver","production_countries",silver_folder_path,merge_condition,"file_date")

In [0]:
%sql
SELECT COUNT(1),file_date
FROM movie_silver.production_countries
GROUP BY file_date

In [0]:
dbutils.notebook.exit("Success")